In [1]:
from datasets import load_dataset, Features, Value

correct_with_label_cleaned_dataset_path = './datasets/correct_with_label_cleaned_camp_data.jsonl'

_features = Features({
    'reason': Value('string'),
    'diseases': Value('string'),
    'feature_content': Value('string')
})

formatted_dataset = load_dataset(
    "json", 
    data_files=correct_with_label_cleaned_dataset_path, 
    split="train", 
    features=_features,
)
formatted_dataset

Dataset({
    features: ['reason', 'diseases', 'feature_content'],
    num_rows: 5000
})

In [2]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
# ### Instruction:
# {}
# ### Input:
# {}
# ### Response:
# {}"""
def format_to_alpaca(example):
    instruction = "你是一位经验丰富的临床医学专家，对各种疾病的症状、体征、检查结果和诊断标准有着深入的了解。同时，你擅长从病历中提取关键信息，运用专业知识进行综合分析，为疾病的诊断提供准确的依据。"
    input_text = example['feature_content']
    diseases_text = example['diseases']
    reason_text = example['reason']
    output_text = """
    患者可能罹患的疾病是{}
    依据如下:
    {}
    """.format(diseases_text, reason_text)
    return {"instruction": instruction, "input": input_text, "output": output_text}

alpaca_formatted_dataset = formatted_dataset.map(format_to_alpaca, remove_columns=['reason', 'diseases', 'feature_content'])
alpaca_formatted_dataset[0]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

{'instruction': '你是一位经验丰富的临床医学专家，对各种疾病的症状、体征、检查结果和诊断标准有着深入的了解。同时，你擅长从病历中提取关键信息，运用专业知识进行综合分析，为疾病的诊断提供准确的依据。',
 'input': '性别: 女\n年龄: 54\n主诉: 糖尿病多年\n现病史: 病人糖尿病多年，服用二甲双胍治疗，血糖控制不佳，建议调整治疗\n既往史: 无特殊\n个人史: \n过敏史: 无\n婚育史: \n流行病史: □1、发病前14天内有病例报告社区的旅行史或居住史；□2、发病前14天内与新型冠状病毒感染的患者或无症状感染者有接触史；□3、发病前14天内曾接触过来自有病例报告社区的发热或有呼吸道症状的患者；□4、聚集性发病（2周内在小范围如家庭、办公室、学校班级等场所，出现2例及以上发热和/或呼吸道症状的病例）；√5、无上述流行病学史。\n体格检查: 血压（收缩压/舒张压）130/80mmHg  \n  空腹血糖测定值：mmol/L  \n  餐后血糖测定值：mmol/L  \n  T：℃  \n  P：次/分  \n  R：次/分  \n  神志：   \n  呼吸：  \n  \n辅助检查: 暂无',
 'output': '\n    患者可能罹患的疾病是2型糖尿病\n    依据如下:\n    1. 患者为54岁女性，主诉糖尿病多年，目前服用二甲双胍治疗，但血糖控制不佳；\n2. 现病史表明患者糖尿病需调整治疗方案，但未提供空腹血糖和餐后血糖具体数值；\n3. 无其他特殊既往史、过敏史或流行病学史；\n4. 体格检查中血压正常，其他生命体征无异常记录；\n5. 目前需进一步评估血糖控制情况，考虑调整治疗方案。\n    '}

In [4]:
alpaca_train_dataset_path = './datasets/alpaca_train_dataset.jsonl'
alpaca_formatted_dataset.to_json(alpaca_train_dataset_path, force_ascii=False)
print(f"Alpaca 格式化数据集已保存到: {alpaca_train_dataset_path}")
alpaca_formatted_dataset[0]

Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Alpaca 格式化数据集已保存到: ./datasets/alpaca_train_dataset.jsonl


{'instruction': '你是一位经验丰富的临床医学专家，对各种疾病的症状、体征、检查结果和诊断标准有着深入的了解。同时，你擅长从病历中提取关键信息，运用专业知识进行综合分析，为疾病的诊断提供准确的依据。',
 'input': '性别: 女\n年龄: 54\n主诉: 糖尿病多年\n现病史: 病人糖尿病多年，服用二甲双胍治疗，血糖控制不佳，建议调整治疗\n既往史: 无特殊\n个人史: \n过敏史: 无\n婚育史: \n流行病史: □1、发病前14天内有病例报告社区的旅行史或居住史；□2、发病前14天内与新型冠状病毒感染的患者或无症状感染者有接触史；□3、发病前14天内曾接触过来自有病例报告社区的发热或有呼吸道症状的患者；□4、聚集性发病（2周内在小范围如家庭、办公室、学校班级等场所，出现2例及以上发热和/或呼吸道症状的病例）；√5、无上述流行病学史。\n体格检查: 血压（收缩压/舒张压）130/80mmHg  \n  空腹血糖测定值：mmol/L  \n  餐后血糖测定值：mmol/L  \n  T：℃  \n  P：次/分  \n  R：次/分  \n  神志：   \n  呼吸：  \n  \n辅助检查: 暂无',
 'output': '\n    患者可能罹患的疾病是2型糖尿病\n    依据如下:\n    1. 患者为54岁女性，主诉糖尿病多年，目前服用二甲双胍治疗，但血糖控制不佳；\n2. 现病史表明患者糖尿病需调整治疗方案，但未提供空腹血糖和餐后血糖具体数值；\n3. 无其他特殊既往史、过敏史或流行病学史；\n4. 体格检查中血压正常，其他生命体征无异常记录；\n5. 目前需进一步评估血糖控制情况，考虑调整治疗方案。\n    '}